In [8]:
import requests
import json
from dotenv import load_dotenv
import os


load_dotenv()

def fetch_traffic_reports():
    # URL API
    url = 'https://api.navitia.io/v1/coverage/fr-idf/line_reports'

    # clé api from env
    api_key = os.getenv('token_navitia')
    headers = {
        'Authorization': api_key
    }
    
    # requete
    response = requests.get(url, headers=headers)

  
    if response.status_code == 200:
       
        data = response.json()
        # ecriture requete positif en hson
        with open('line_reports.json', 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print("Data written to line_reports.json")
    else:
        
        print('Failed to retrieve data: ', response.status_code)

# Appel func
fetch_traffic_reports()


Data written to line_reports.json


In [12]:
import pandas as pd


with open('line_reports.json') as f:
    data = json.load(f)

# Further normalize the 'messages' field from the 'disruptions' section
messages = pd.json_normalize(
    data, 
    record_path=['disruptions', 'messages'], 
    meta=[['disruptions', 'id'], ['disruptions', 'disruption_id']],
    errors='ignore'
)

# Display the 'messages' DataFrame
messages.head()


messages


,feed_publishers,disruptions,line_reports,warnings,links,pagination.total_result,pagination.start_page,pagination.items_per_page,pagination.items_on_page,context.current_datetime,context.timezone
0,"[{'id': 'osm', 'name': 'openstreetmap', 'url':...",[{'id': 'e9cbe72a-04c5-11ef-9db7-0a58a9feac02'...,"[{'line': {'id': 'line:IDFM:C00563', 'name': '...","[{'id': 'beta_endpoint', 'message': 'This serv...",[{'href': 'https://api.navitia.io/v1/coverage/...,1048,0,25,25,20240430T052303,Europe/Paris
